In [2]:
import pandas as pd
import numpy as np
import os
os.getcwd()

'c:\\Users\\naima\\Desktop\\10Academy\\KAIM8\\fraud-detection\\notebooks'

In [3]:
fraud_df = pd.read_csv("../data/raw/Fraud_Data.csv")
fraud_df = pd.read_csv("../data/raw/IpAddress_to_Country.csv")
fraud_df.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


In [4]:
fraud_df.columns = fraud_df.columns.str.strip().str.lower()

In [5]:
fraud_df.columns

Index(['lower_bound_ip_address', 'upper_bound_ip_address', 'country'], dtype='object')

In [6]:
fraud_df = pd.read_csv("../data/raw/Fraud_Data.csv")

# This is the IP-to-country mapping
ip_df = pd.read_csv("../data/raw/IpAddress_to_Country.csv")

# Optional: clean column names
fraud_df.columns = fraud_df.columns.str.strip().str.lower()
ip_df.columns = ip_df.columns.str.strip().str.lower()

# Quick check
fraud_df.columns

Index(['user_id', 'signup_time', 'purchase_time', 'purchase_value',
       'device_id', 'source', 'browser', 'sex', 'age', 'ip_address', 'class'],
      dtype='object')

In [7]:
# Lowercase and remove spaces
fraud_df.columns = fraud_df.columns.str.strip().str.lower()
ip_df.columns = ip_df.columns.str.strip().str.lower()

In [8]:
fraud_df['ip_address'] = fraud_df['ip_address'].astype(int)
ip_df['lower_bound_ip_address'] = ip_df['lower_bound_ip_address'].astype(int)
ip_df['upper_bound_ip_address'] = ip_df['upper_bound_ip_address'].astype(int)

In [9]:
# Sort before merge_asof
fraud_df = fraud_df.sort_values('ip_address')
ip_df = ip_df.sort_values('lower_bound_ip_address')

fraud_df = pd.merge_asof(
    fraud_df,
    ip_df,
    left_on='ip_address',
    right_on='lower_bound_ip_address',
    direction='backward'
)

# Keep only rows where IP is within the bounds
fraud_df = fraud_df[fraud_df['ip_address'] <= fraud_df['upper_bound_ip_address']]

In [10]:
fraud_df[['ip_address', 'country']].head()

,ip_address,country
634,16778864,Australia
635,16842045,Thailand
636,16843656,China
637,16938732,China
638,16971984,Thailand


In [11]:


fraud_df['signup_time'] = pd.to_datetime(fraud_df['signup_time'])
fraud_df['purchase_time'] = pd.to_datetime(fraud_df['purchase_time'])



In [12]:

fraud_df['time_since_signup'] = (
    fraud_df['purchase_time'] - fraud_df['signup_time']
).dt.total_seconds()



In [13]:
fraud_df['hour_of_day'] = fraud_df['purchase_time'].dt.hour
fraud_df['day_of_week'] = fraud_df['purchase_time'].dt.dayofweek

In [14]:
fraud_df = fraud_df.sort_values(['user_id', 'purchase_time'])

fraud_df['transactions_per_user'] = (
    fraud_df.groupby('user_id').cumcount() + 1
)

In [15]:

fraud_df.to_csv("../data/raw/Fraud_Data.csv", index=False)


In [16]:
fraud_df['class'].value_counts(normalize=True)

class
0    0.905007
1    0.094993
Name: proportion, dtype: float64

In [17]:
# Make sure this is your final processed DataFrame
fraud_df.to_csv("../data/processed/fraud_processed.csv", index=False)

